In [1]:

from helper_code import *
import numpy as np, os, sys, joblib
import ecg_plot
import pandas as pd
from glob import glob
import os
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from tsai.all import *
import torch
from sklearn.metrics import classification_report
import transformation_funcs as tfs
import seaborn as sns
from torchsummary import summary
torch.cuda.set_device(2) 

/home/amirsalimi/miniconda3/envs/ecg_augment/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [28]:

# DATASET_NAME = "WFDB_CPSC2018_signitured"
DATASET_NAME = "WFDB_ChapmanShaoxing_signitured"
X = np.load('./data/big_numpy_datasets/%s.npy'%DATASET_NAME, mmap_mode='c')
label_df = pd.read_csv("data/%s.csv"%DATASET_NAME).drop(columns=["headers","leads"])
y = snomedConvert(label_df)
# get diseases that exist in more than 5 percent of the samples
y=y[y.columns[y.sum()>(0.05*y.shape[0])]]
y.reset_index(drop=True)
X.shape,y.shape

((10247, 12, 5000), (10247, 8))

In [29]:
# more than 1 label, 0 labels, total num labels, 
np.sum(y.sum(axis=1)>0),np.sum(y.sum(axis=1)==0),y.sum().sum()

(9910, 337, 13979)

In [34]:

DATASET_NAME = "WFDB_CPSC2018_signitured"
X = np.load('./data/big_numpy_datasets/%s.npy'%DATASET_NAME, mmap_mode='c')
label_df = pd.read_csv("data/%s.csv"%DATASET_NAME).drop(columns=["headers","leads"])
y = snomedConvert(label_df)
# get diseases that exist in more than 5 percent of the samples
y=y[y.columns[y.sum()>(0.05*y.shape[0])]]
y.reset_index(drop=True)
X.shape,y.shape

((6877, 12, 14000), (6877, 7))

In [37]:
# more than 1 label, 0 labels, total num labels, 
np.sum(y.sum(axis=1)>1),np.sum(y.sum(axis=1)==0),y.sum().sum()

(394, 368, 6903)

In [36]:
print(y.sum().to_latex())

\begin{tabular}{lr}
\toprule
{} &     0 \\
\midrule
right bundle branch block    &  1857 \\
ventricular ectopics         &   700 \\
atrial fibrillation          &  1221 \\
1st degree av block          &   722 \\
premature atrial contraction &   616 \\
sinus rhythm                 &   918 \\
st depression                &   869 \\
\bottomrule
\end{tabular}



In [13]:

DATASET_NAME = "WFDB_PTBXL_signitured"
X = np.load('./data/big_numpy_datasets/%s.npy'%DATASET_NAME, mmap_mode='c')
label_df = pd.read_csv("data/%s.csv"%DATASET_NAME).drop(columns=["headers","leads"])
y = snomedConvert(label_df)
# get diseases that exist in more than 5 percent of the samples
y=y[y.columns[y.sum()>(0.05*y.shape[0])]]
y.reset_index(drop=True)
X.shape,y.shape

((21837, 12, 5000), (21837, 11))

In [18]:
# more than 1 label, 0 labels, total num labels, 
np.sum(y.sum(axis=1)>1),np.sum(y.sum(axis=1)==0),y.sum().sum()

(12093, 526, 44179)

In [15]:
print(y.sum().to_latex())

\begin{tabular}{lr}
\toprule
{} &      0 \\
\midrule
left axis deviation                  &   5146 \\
myocardial ischemia                  &   2175 \\
myocardial infarction                &   5261 \\
left ventricular hypertrophy         &   2359 \\
ventricular ectopics                 &   1154 \\
atrial fibrillation                  &   1514 \\
t wave abnormal                      &   2345 \\
abnormal QRS                         &   3389 \\
sinus rhythm                         &  18092 \\
left anterior fascicular block       &   1626 \\
incomplete right bundle branch block &   1118 \\
\bottomrule
\end{tabular}



In [16]:
y

,left axis deviation,myocardial ischemia,myocardial infarction,left ventricular hypertrophy,ventricular ectopics,atrial fibrillation,t wave abnormal,abnormal QRS,sinus rhythm,left anterior fascicular block,incomplete right bundle branch block
0,False,False,False,False,False,False,False,False,True,False,False
1,False,False,False,False,False,False,False,False,True,False,False
2,False,False,False,False,False,False,False,False,True,False,False
3,False,False,False,False,False,False,False,False,True,False,False
4,False,False,False,False,False,False,False,False,True,False,False
...,...,...,...,...,...,...,...,...,...,...,...
21832,True,False,False,True,True,False,True,False,False,False,False
21833,False,False,False,False,False,False,False,True,True,False,False
21834,False,True,False,False,False,False,False,False,True,False,False
21835,True,False,False,False,False,False,False,False,True,False,False


Table of past-work (Hong et al.)

In [91]:
df = pd.read_csv("csvs/physionet-2020-survey Hongetal.csv")[0:5]
top_df = df[["Rank","Data preprocessing","Data augmentation"]]
print(top_df.set_index("Rank").to_latex())

\begin{tabular}{lll}
\toprule
{} &                                                                 Data preprocessing &                                                                          Data augmentation \\
Rank &                                                                                    &                                                                                            \\
\midrule
1    &  Sampling rate 500Hz; Bandwidth 3 - 45 Hz; Value -1 - 1; Window size 15s 0-padding &                                                                                         No \\
2    &                Sampling rate 257 Hz; Value -1 - 1; Window size 4096(16s) 0-padding &                                                                                         No \\
3    &   Sampling rate 500Hz; Window size 30s 0-padding; Exclude 4 leads; Wavelet denoise &                                                                  With external data: Hefei \\
4    &                     Sampli